# Daten klassifizieren

**Inhalt:** Unsaubere Daten laden und klassifizieren

**Nötige Skills:** Erste Schritte mit Pandas

**Lernziele:**
- Daten auf Integrität prüfen
- Einfaches Putzen der gröbsten Fehler
- Ein paar String-Funktionen
- Klassifizieren a: df.apply kennenlernen
- Klassifizieren b: df.merge kennenlernen
- Plotting Level 2: mehrere Serien

# Das Beispiel

P3-Datenbank des Schweizerischen Nationalfonds. Beinhaltet alle Forschungsprojekte, die seit 1975 vom SNF Fördergelder erhalten haben.

Quelle und Dokumentation: http://p3.snf.ch/Pages/DataAndDocumentation.aspx

Datenfile: http://p3.snf.ch/P3Export/P3_GrantExport.csv

Speichern Sie die Datei an einem geeigneten Ort, zB im Unterornder `dataprojects/SNF/`

## Vorbereitung

Wir laden diesmal nicht nur das Pandas-Modul, sondern auch NumPy.

*NumPy is the fundamental package for scientific computing with Python): http://www.numpy.org/*

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline

## Datenfile laden

Wie gehabt ... Achtung, die Felder sind nicht durch Kommas sondern durch Semikolon abgetrennt!

In [ ]:
path = "dataprojects/SNF/P3_GrantExport.csv"

In [ ]:
df = pd.read_csv(path, delimiter=';')

In [ ]:
df.head(2)

Besser! Schauen wir uns die Sache mal näher an.

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

Offensichtlich hat es einige Spalten drin, die noch nicht mit dem richtigen Datentyp formatiert sind, z.B. "Approved Amount".

Das Problem ist: So lange wir da nicht die richtigen Datentypen haben, funktionieren einige Auswertungen nicht.

In [ ]:
#Zum Beispiel diese hier:
df['Approved Amount'].mean()

Eigentlich wären das sehr interessante Informationen: wie viel Geld haben die Projekte im Schnitt gekriegt, im Maximum, im Minimum, etc.

## Entfernen von ungültigen Werten

Wir müssen also irgendwie diese Spalte reinigen, damit Pandas die Berechungen für uns machen kann.

Um herauszukriegen, was das Problem sein könnte, ist `value_counts()` eine ziemlich einfache Option.

In [ ]:
df['Approved Amount'].value_counts().sort_index()

Das Problem liegt in der letzten Zeile: Bei 12070 Einträgen steht: "`data not included in P3`."

Wir können das auf mehrere Arten lösen:

### Variante 1: Werte mit NaN ersetzen

Wir verwenden nun die Funktion `replace()`, um selektiv alle Instanzen von "`data not included in P3`" zu ersetzen - und zwar mit NaN:

In [ ]:
df['Approved Amount'] = df['Approved Amount'].replace('data not included in P3', np.nan)

Die Einträge wurden jetzt in NaN verwandelt (und werden deshalb standardmässig gar nicht mehr angezeigt)

In [ ]:
df['Approved Amount'].value_counts().sort_index()

Allerdings haben wir ein Problem: Der Datentyp von "Approved Amount" ist immer noch "object"...

In [ ]:
df.dtypes

Wir sind gezwungen, noch eine Datenkonversion durchzuführen: mit `astype()`

In [ ]:
df['Approved Amount'] = df['Approved Amount'].astype(float)

Endlich stimmt der Datentyp:

In [ ]:
df.dtypes

Und wir können unsere Auswertung durchführen:

In [ ]:
#Antwort
df['Approved Amount'].mean()

### Variante 2: Datei nochmals einlesen mit einer Spezialoption

Um uns einige Schritte zu ersparen, lesen wir die Datei einfach nochmals neu ein.

Die Option heisst `na_values=` (na = Not Available, wird durch NaN ersetzt = Not a Number oder so)

In [ ]:
df = pd.read_csv(path, delimiter=';', na_values='data not included in P3')

Tadaaa!

In [ ]:
df.dtypes

**Übrigens:** Um zu checken, was es eigentlich mit den ungültigen Einträgen eigentlich auf sich hat, können wir `.isnull()` verwenden:

In [ ]:
df[df['Approved Amount'].isnull()]

Es scheint sich also hier um ein spezielles Förderinstrument zu handeln ("Fellowships").

### Etwas Repetition vom Tag 1...

**Quiz:** Was war der maximale Betrag, den ein Projekt erhielt? Das Minimum? Der Median?

In [ ]:
#Antwort


In [ ]:
#Antwort


In [ ]:
#Antwort


**Quiz:** Suche die fünfzig Projekte raus, die am meisten Geld gekriegt haben. Welche Universitäten kommen darunter am meisten vor?

In [ ]:
#Antwort


**Quiz:** Über welches Förderinstrument ("Funding Instrument Hierarchy") wurde insgesamt am meisten Geld vergeben?

In [ ]:
#Antwort


**Quiz:** Stellen Sie die Verteilung sämtlicher gesprochenen Beträge in einem Histogramm dar!

In [ ]:
#Antwort


**Quiz:** In welchen Ländern waren die vergebenen Beträge im Schnitt am Grössten? Zeigen Sie die zehn obersten an.

In [ ]:
#Antwort


In [ ]:
# Time for a break ...

## Werte Kategorisieren

Sagen wir mal, wir interessieren uns für die Institutionen in der Schweiz, die vom SNF Geld gekriegt haben.

Wir erstellen erstmal ein Dataframe, in dem nur diese Institutionen vorkommen:

In [ ]:
df_swiss = df[df['Institution Country'] == 'Switzerland']

Und lassen uns dann eine Liste aller Universitäten anzeigen, die in diesem Dataframe vorkommen:

In [ ]:
df_swiss['University'].unique()

Schnell wird klar: In dieser Liste sind nicht nur Universitäten, sondern auch Fachhochschulen und andere Institutionen enthalten.

Wie gehen wir vor, wenn wir die Daten nach diesen Typen klassifizieren wollen? Mit anderen Worten, zB separate Durchschnittswerte ausrechnen für Universitäten, Fachhochschulen, etc?

### Methode 1: contains, replace

Die allereinfachste (und nicht sehr empfehlenswerte) Variante ist, einfach zu checken, ob in einem bestimmten Eintrag das Wort "University" vorkommt.

Wir können dafür die Funktion `str.contains()` verwenden - heraus kommt eine Liste von True/False-Werten, die wir weiter verwenden können...

In [ ]:
df_swiss['University'].str.contains('University')

Zum Beispiel so:

In [ ]:
df_swiss['Institution Type'] = df_swiss['University'].str.contains('University')

... oder vielleicht doch nicht so :-) Der Grund für die obige Warnung ist: Wir arbeiten auf einem Slice eines Dataframes, das kann Probleme machen (muss aber nicht).

Um sicher zu sein: `.copy()` verwenden, um im Memory eine physische Kopie des Dataframes zu erstellen

In [ ]:
df_ch = df_swiss.copy()

In [ ]:
df_ch['Institution Type'] = df_ch['University'].str.contains('University')

In [ ]:
df_ch.head(3)

Nun können wir die True/False-Werte mit generischen Einträgen ersetzen. Dafür gibt es `replace()`:

In [ ]:
df_ch['Institution Type'] = df_ch['Institution Type'].replace(True, 'University')

In [ ]:
df_ch['Institution Type'] = df_ch['Institution Type'].replace(False, 'Other')

In [ ]:
df_ch.head(3)

Wir können nun zB ausrechnen, wie viel Geld die Universitäten und die übrigen Institutionen in der Summe gekriegt haben:

In [ ]:
df_ch.groupby('Institution Type')['Approved Amount'].sum()

Aber wie gesagt, es gibt bessere Wege. (zB haben wir nun Einträge wie "Université" nicht berücksichtigt.

### Methode 2: apply, isin

Auch nicht wirklich super, aber immerhin besser als vorher: Wir schreiben eine eigene Funktion zur Klassifizierung von Universitäten.

Diese Funktion können wir unendlich kompliziert machen, wenn wir wollen. Hier halten wir sie bewusst einfach.

In [ ]:
def categorize_institution(institution):
    
    #Ist eine Institution eine Uni? Hier eine Liste von Wörtern, nach denen wir suchen.
    university_names = ["University", "Universität", "Université"]
    
    #Gehen wir die Liste durch...
    for university_name in university_names:
        
        #Kommt das Wort im String, den wir klassifizieren wollen, mehr als null mal vor?
        if str(institution).count(university_name) > 0:
            
            #Dann ist es eine Universität
            return "University"
    
    #sonst nicht
    return "Other"

Wir testen die Funktion...

In [ ]:
categorize_institution("University of Zurich")

In [ ]:
categorize_institution("Fachhochschule Nordwestschweiz")

... und wenden sie auf die Spalte "University" an.

In [ ]:
df_ch['University'].apply(categorize_institution)

Das Resultat kommt nun in die Spalte "Institution Type"

In [ ]:
df_ch['Institution Type'] = df_ch['University'].apply(categorize_institution)

In [ ]:
df_ch.head(3)

Wir sind jetzt ziemlich schnell durch `df.apply()` durchgegangen. Macht nix, wir kommen später nochmals drauf zurück. Man kann die Funktion übrigens auch auf ganze Zeilen anwenden, mehr dazu später.

**Quiz:** Basierend auf unserer neuen Klassifizierung: Zeichnen Sie einen Balkenchart, der die durchschnittliche Vergabesumme für Universitäten und Nicht-Universitäten anzeigt.

In [ ]:
#Antwort


### Methode 3: merge

Und nun zur saubersten Art, wie man die Institutionen in dieser Tabelle hier klassifizieren sollte: von Hand.

Wie ziehen uns nochmals die Liste der unique Values, diesmal gleich als Dataframe:

In [ ]:
df_unique = pd.DataFrame(df_ch['University'].unique())
df_unique

Weil es einfacher geht, bearbeiten wir die Liste in einem externen Programm... mit der Funktion `to_csv()`

In [ ]:
df_unique.to_csv('dataprojects/SNF/klassifizieren.csv', index=False)

... im Excel, oder anderswo bearbeiten, und wieder laden: (Ich habe das hier schonmal vorbereitet)

In [ ]:
df_unique_edited = pd.read_csv('dataprojects/SNF/klassifiziert.csv')

In [ ]:
df_unique_edited

Wir haben jetzt zwei Tabellen: `df_ch` (die grosse Datentabelle) und `df_unique` (die Klassifizierungen).

Diese zwei Tabellen können wir nun verknüpfen, und zwar mit der Funktion `merge()`

In [ ]:
df_ch_classified = df_ch.merge(df_unique_edited, how='left', left_on='University', right_on='University')
df_ch_classified

Die Spalte "New Type" wurde nun zur Tabelle "df_ch" hinzugefügt, und zwar genau dort, wo es zum Eintrag in "University" passt!

Schauen wir kurz, wie viele Einträge es von welchem Typ hat:

In [ ]:
df_ch_classified['New Type'].value_counts()

Ging auch wirklich nichts vergessen?

In [ ]:
df_ch_classified['New Type'].value_counts(dropna=False)

**Oops!** Es hat einen fehlenden Eintrag drin.

Was ist das für ein Eintrag?

In [ ]:
df_ch_classified[df_ch_classified['New Type'].isnull()]

Sieht nach einem grundsätzlich validen Projekt aus. Wir klassifizieren diesen Eintrag kurzerhand auf "Other":

In [ ]:
df_ch_classified.loc[24179, "New Type"] = "Other"

In [ ]:
df_ch_classified.loc[24179]

**Quiz:** Kategorisieren Sie die Einträge nach dem Herkunftsland der Universität (erstellen Sie dazu ein neues Feld "Country Type" mit den Einträgen "Switzerland" oder "Other". Wie viele Projekte kommen aus der Schweiz, wie viele aus anderen Ländern? (wir wenden hier die Methode 1 an)

**Achtung** Wechseln Sie jetzt wieder zum originalen Dataframe, "df"

In [ ]:
#Neues, leeres Feld 'Country Type' erstellen


In [ ]:
# Country Type = 'Switzerland', falls Switzerland


In [ ]:
# Country Type = 'Other', falls nicht


In [ ]:
# Auswertung nach Country Type


## Plotting Level 2

Nun wollen wir darstellen, wie sich die Projekte über die Zeit hinweg in der Schweiz und in den übrigen Ländern entwickelt haben. Es geht also darum, zwei verschiedene Serien auf einer Grafik einzuzeichnen.

Wir wenden dazu jetzt einen etwas faulen Trick an, um eine neue Spalte mit dem Jahr zu generieren (eigentlich gäbe es dazu noch einen speziellen Datentyp, aber den schauen wir ein anderes Mal an).

In [ ]:
df['Year'] = df['Start Date'].str[6:]

Check, ob das einigermassen funktioniert hat...

In [ ]:
df['Year'].value_counts(dropna=False).sort_index()

Jetzt plotten wir die Gesamtsumme der gesprochenen Gelder nach Jahr. Zuerst für die Schweiz ...

In [ ]:
df[df['Country Type'] == "Switzerland"].groupby('Year')['Approved Amount'].sum().plot(figsize=(12,6))

... dann für die anderen Länder ...

In [ ]:
df[df['Country Type'] == "Other"].groupby('Year')['Approved Amount'].sum().plot(figsize=(12,6))

... und schliesslich für beide Ländertypen:

### Methode 1: Zwei verschiedene Linien einzeichnen

Die sicherste Methode, um mehrere Kurven auf derselben Grafik darzustellen, ist `ax=`.

Wir speichern erste einen Plot als "chart1" und sagen dem zweiten Plot dann, sich zu "chart1" hinzuzugesellen.

In [ ]:
chart1 = df[df['Country Type'] == "Switzerland"].groupby('Year')['Approved Amount'].sum().plot(figsize=(12,6))

df[df['Country Type'] == "Other"].groupby('Year')['Approved Amount'].sum().plot(ax=chart1)

### Methode 2: Doppelt groupby, unstack

In diesem Fall gibt es allerdings noch eine elegantere Variante. Und zwar mit `groupby()`.

Diese Methode funktioniert nicht nur mit einem Level, sondern auch mit zwei. Die Summierung wird einerseits über die Jahre ("Years") gemacht und andererseits für die einzelenen Ländertypen ("Country Types"):

In [ ]:
df.groupby(['Year', 'Country Type'])['Approved Amount'].sum()

Um diese Werte zu plotten, müssen wir Pandas die Tabelle allerdings etwas anders zur Verfügung stellen: im Wide-Format (dazu später noch mehr). Wir können dazu die Funktion `unstack()` verwenden:

In [ ]:
df.groupby(['Year', 'Country Type'])['Approved Amount'].sum().unstack()

Letzter Schritt: `plot()`

In [ ]:
df.groupby(['Year', 'Country Type'])['Approved Amount'].sum().unstack().plot(figsize=(12,6))

**Quiz:** Plotten Sie den durchschnittlichen Betrag, den Universitäten, Fachhochschulen, Spitäler und andere Institutionen über die Jahre erhalten haben - alles auf derselben Grafik. Benutzen Sie dazu wieder das Dataframe "df_ch_classfied" – Achtung, Sie müssen zuerst wieder eine Jahresspalte erstellen.

In [ ]:
# Spalte 'Year' in df_ch_classified erstellen


In [ ]:
# Liste, nach Jahr und New Type gruppiert


In [ ]:
# Plot


**Schlussfrage:** Haben wir nun bereits eine Story gefunden? Wenn ja, was könnte sie sein? Wenn nein, welches wären weitere Auswertungen, die man basierend auf diesen Daten machen könnte?

In [ ]:
#Antwort in Textform...
#Zum Beispiel: Auswertung der Profile von einzelnen Forschern.

# Übung

Wir klassifizieren die Projekte nun nach Forschungsdisziplin und werten aus, welche Disziplinen zu welchem Zeitpunkt wie viel Geld gekriegt haben (wir wenden dabei Methode 3 an).

**Schritt 1:** Wir erstellen eine Liste der einzigartigen Einträge im Datenfeld "Discipline Name" und speichern sie als csv-Datei ab. (Arbeiten Sie mit dem dataframe "df_ch")

In [ ]:
#Dataframe aus einzigartigen Disziplinennamen erstellen


In [ ]:
# Dataframe als csv speichern


**Schritt 2:** Wir bearbeiten das csv-File extern und klassifizieren nach unserer Wahl

In [ ]:
#extern bearbeiten...

**Schritt 3:** Wir fügen die Klassifizierung der Disziplinen in unsere Datenliste (Arbeiten Sie mit df) ein

In [ ]:
# Einlesen des bearbeiteten csv-Files


In [ ]:
# Verbinden Sie das dataframe "df_ch" mit der Klassifizierung, abspeichern unter neuem dataframe df_ch_classified


**Schritt 4:** Auswertungen

- Wie viele Projekte von welchem Disziplinen-Typ wurden durchgeführt?

- Welche Disziplinen-Typen haben meisten Geld gekriegt?

- Wie viel kosten Projekte der Disziplinen-Typen im Durchschnitt? Im Median?

**Schritt 5:** Plot einer Auswertung

Wie viel Geld haben die verschiedenen Disziplinentypen im Jahresverlauf insgesamt gekriegt?

In [ ]:
#Wir müssen auf df_ch_classified nochmals den Trick mit der Jahresspalte anwenden


In [ ]:
# Tabelle anzeigen: Summe der gesprochenen Gelder, gruppiert nach Jahr und Disziplinentyp


In [ ]:
#Plot als Liniendiagramm
